In [2]:
import pandas as pd
import numpy as np
import pprint
import itertools
import random
from collections import Counter as ctr
from sodapy import Socrata
import geopy
from geopy.geocoders import Nominatim

In [3]:
client_choques_nyc = Socrata('data.cityofnewyork.us',
                  'P5yEsPDgBpkSlfs3G7Kg0DM6o',)

client_robo = Socrata('data.cityofnewyork.us',
                  'P5yEsPDgBpkSlfs3G7Kg0DM6o',)

client_edad = Socrata('data.cityofnewyork.us',
                  'P5yEsPDgBpkSlfs3G7Kg0DM6o',)

In [4]:
results_choques_nyc = client_choques_nyc.get("h9gi-nx95", limit=800000)
results_df_choques_nyc = pd.DataFrame.from_records(results_choques_nyc)

results_robo = client_robo.get("5uac-w243", limit=258000)
results_df_robo = pd.DataFrame.from_records(results_robo)

results_edad = client_edad.get("bm4k-52h4", limit=1000000)
results_df_edad = pd.DataFrame.from_records(results_edad)

In [5]:
results_df_choques_nyc['crash_date'].replace({'T00:00:00.000':''}, regex=True, inplace=True)
results_df_choques_nyc['crash_date_time']=pd.to_datetime(results_df_choques_nyc['crash_date'] + results_df_choques_nyc['crash_time'], format='%Y-%m-%d%H:%M')

In [6]:
choques_nyc=results_df_choques_nyc[results_df_choques_nyc['crash_date_time'].dt.year.isin([2019,2020,2021,2022])]

In [7]:
#traduccion de columnas
choques_nyc.rename(columns= {'crash_date_time': 'colision_fecha_hora',
 'crash_date': 'colision_fecha',
 'crash_time': 'colision_hora',
 'on_street_name': 'nombre_de_calle',
 'off_street_name': 'nombre_fuera_de_calle',
 'cross_street_name': 'calle_interseccion',
 'number_of_persons_injured': 'numero_de_personas_heridas',
 'number_of_persons_killed': 'numero_de_personas_fallecidas',
 'number_of_pedestrians_injured': 'numero_de_peatones_heridos',
 'number_of_pedestrians_killed': 'numero_de_peatones_fallecidos',
 'number_of_cyclist_injured': 'numero_de_ciclistas_heridos',
 'number_of_cyclist_killed': 'numero_de_ciclistas_fallecidos',
 'number_of_motorist_injured': 'numero_de_conductores_heridos',
 'number_of_motorist_killed': 'numero_de_conductores_fallecidos',
 'contributing_factor_vehicle_1': 'factor_contribuyente_vehiculo_1',
 'contributing_factor_vehicle_2': 'factor_contribuyente_vehiculo_2',
 'collision_id': 'colision_ID',
 'vehicle_type_code1': 'codigo_tipo_de_vehiculo_1',
 'vehicle_type_code2': 'codigo_tipo_de_vehiculo_2',
 'borough': 'localidad',
 'zip_code': 'codigo_postal',
 'latitude': 'latitud',
 'longitude': 'longitud',
 'location': 'ubicacion',
 'contributing_factor_vehicle_3': 'factor_contribuyente_vehiculo_3',
 'vehicle_type_code_3': 'codigo_tipo_de_vehiculo_3',
 'contributing_factor_vehicle_4': 'factor_contribuyente_vehiculo_4',
 'vehicle_type_code_4': 'codigo_tipo_de_vehiculo_4',
 'contributing_factor_vehicle_5': 'factor_contribuyente_vehiculo_5',
 'vehicle_type_code_5': 'codigo_tipo_de_vehiculo_5',
 'day': 'dia',
 'no_location': 'sin_ubicacion',
 'day_time': 'hora_del_dia'}, inplace= True)
choques_nyc.head(1)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\756886217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.rename(columns= {'crash_date_time': 'colision_fecha_hora',


,colision_fecha,colision_hora,latitud,longitud,ubicacion,nombre_de_calle,numero_de_personas_heridas,numero_de_personas_fallecidas,numero_de_peatones_heridos,numero_de_peatones_fallecidos,...,codigo_postal,calle_interseccion,nombre_fuera_de_calle,factor_contribuyente_vehiculo_3,factor_contribuyente_vehiculo_4,codigo_tipo_de_vehiculo_3,codigo_tipo_de_vehiculo_4,factor_contribuyente_vehiculo_5,codigo_tipo_de_vehiculo_5,colision_fecha_hora
0,2022-07-20,1:25,40.835808,-73.89083,"{'latitude': '40.835808', 'longitude': '-73.89...",BOSTON ROAD,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-20 01:25:00


In [8]:
choques_nyc.columns

Index(['colision_fecha', 'colision_hora', 'latitud', 'longitud', 'ubicacion',
       'nombre_de_calle', 'numero_de_personas_heridas',
       'numero_de_personas_fallecidas', 'numero_de_peatones_heridos',
       'numero_de_peatones_fallecidos', 'numero_de_ciclistas_heridos',
       'numero_de_ciclistas_fallecidos', 'numero_de_conductores_heridos',
       'numero_de_conductores_fallecidos', 'factor_contribuyente_vehiculo_1',
       'factor_contribuyente_vehiculo_2', 'colision_ID',
       'codigo_tipo_de_vehiculo_1', 'codigo_tipo_de_vehiculo_2', 'localidad',
       'codigo_postal', 'calle_interseccion', 'nombre_fuera_de_calle',
       'factor_contribuyente_vehiculo_3', 'factor_contribuyente_vehiculo_4',
       'codigo_tipo_de_vehiculo_3', 'codigo_tipo_de_vehiculo_4',
       'factor_contribuyente_vehiculo_5', 'codigo_tipo_de_vehiculo_5',
       'colision_fecha_hora'],
      dtype='object')

In [9]:
choques_nyc['anio'] = choques_nyc['colision_fecha_hora'].dt.year
choques_nyc['mes'] = choques_nyc['colision_fecha_hora'].dt.month
choques_nyc['dia'] = choques_nyc['colision_fecha_hora'].dt.day
choques_nyc['hora'] = choques_nyc['colision_fecha_hora'].dt.time

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2988236535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['anio'] = choques_nyc['colision_fecha_hora'].dt.year
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2988236535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['mes'] = choques_nyc['colision_fecha_hora'].dt.month
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2988236535.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [10]:
choques_nyc.columns

Index(['colision_fecha', 'colision_hora', 'latitud', 'longitud', 'ubicacion',
       'nombre_de_calle', 'numero_de_personas_heridas',
       'numero_de_personas_fallecidas', 'numero_de_peatones_heridos',
       'numero_de_peatones_fallecidos', 'numero_de_ciclistas_heridos',
       'numero_de_ciclistas_fallecidos', 'numero_de_conductores_heridos',
       'numero_de_conductores_fallecidos', 'factor_contribuyente_vehiculo_1',
       'factor_contribuyente_vehiculo_2', 'colision_ID',
       'codigo_tipo_de_vehiculo_1', 'codigo_tipo_de_vehiculo_2', 'localidad',
       'codigo_postal', 'calle_interseccion', 'nombre_fuera_de_calle',
       'factor_contribuyente_vehiculo_3', 'factor_contribuyente_vehiculo_4',
       'codigo_tipo_de_vehiculo_3', 'codigo_tipo_de_vehiculo_4',
       'factor_contribuyente_vehiculo_5', 'codigo_tipo_de_vehiculo_5',
       'colision_fecha_hora', 'anio', 'mes', 'dia', 'hora'],
      dtype='object')

In [11]:
#guardo cada columna de codigo de vehiculo en su variable en forma de lista
#y cambio el valor a str por los nan y simbolos especiales
codigo_tipo_de_vehiculo_1 = list(choques_nyc.codigo_tipo_de_vehiculo_1)
codigo_tipo_de_vehiculo_1 = [str(v) for v in codigo_tipo_de_vehiculo_1]
codigo_tipo_de_vehiculo_2 = list(choques_nyc.codigo_tipo_de_vehiculo_2)
codigo_tipo_de_vehiculo_2 = [str(v) for v in codigo_tipo_de_vehiculo_2]
codigo_tipo_de_vehiculo_3 = list(choques_nyc.codigo_tipo_de_vehiculo_3)
codigo_tipo_de_vehiculo_3 = [str(v) for v in codigo_tipo_de_vehiculo_3]
codigo_tipo_de_vehiculo_4 = list(choques_nyc.codigo_tipo_de_vehiculo_4)
codigo_tipo_de_vehiculo_4 = [str(v) for v in codigo_tipo_de_vehiculo_4]
codigo_tipo_de_vehiculo_5 = list(choques_nyc.codigo_tipo_de_vehiculo_5)
codigo_tipo_de_vehiculo_5 = [str(v) for v in codigo_tipo_de_vehiculo_5]

In [12]:
#formateo de primer letra a mayuscula
codigo_tipo_de_vehiculo_1 = [w.title() for w in codigo_tipo_de_vehiculo_1]
codigo_tipo_de_vehiculo_2 = [w.title() for w in codigo_tipo_de_vehiculo_2]
codigo_tipo_de_vehiculo_3 = [w.title() for w in codigo_tipo_de_vehiculo_3]
codigo_tipo_de_vehiculo_4 = [w.title() for w in codigo_tipo_de_vehiculo_4]
codigo_tipo_de_vehiculo_5 = [w.title() for w in codigo_tipo_de_vehiculo_5]

In [13]:
#saque del dataset los modelos de vehiculo mas predominantes
#separando aquellos que tienen apenas apariciones y nada de peso en los numeros
#luego reemplace los nan por unknown
#y aquellos que no forman parte de la lista de los modelos principales
#son reemplazados por uno de esos modelos al azar
#asi me deshago de toda la basura entre los modelos
def normalize_vehicles(abc):
    good_ones = ['Sedan', 'Station Wagon/Sport Utility Vehicle', 'Taxi', 'Pick-Up Truck', 'Box Truck', 'Bike', 'Bus', 'Tractor Truck Diesel', 'Motorcycle', 'Van', 'E-Bike', 'Ambulance', 'E-Scooter', 'Dump', 'Convertible', 'Flat Bed', 'Moped', 'Garbage Or Refuse']
    list_vehicles = [x for x in abc]
    for v in range(0, len(list_vehicles)):
        if list_vehicles[v] == 'Nan':
            list_vehicles[v] = 'Unknown'
        elif list_vehicles[v] not in good_ones:
            list_vehicles[v] = random.choice(good_ones)
    return list_vehicles

In [14]:
#uso la funcion de normalizacion en cada columna de modelo de vehiculo
vehiculo_1 = normalize_vehicles(codigo_tipo_de_vehiculo_1)
vehiculo_2 = normalize_vehicles(codigo_tipo_de_vehiculo_2)
vehiculo_3 = normalize_vehicles(codigo_tipo_de_vehiculo_3)
vehiculo_4 = normalize_vehicles(codigo_tipo_de_vehiculo_4)
vehiculo_5 = normalize_vehicles(codigo_tipo_de_vehiculo_5)

In [15]:
#checkeo que los grupos esten limpios usando counter
vehiculo_3_counter = ctr(vehiculo_3)
vehiculo_3_most_common = vehiculo_3_counter.most_common(50)
print(vehiculo_3_most_common)

[('Unknown', 460565), ('Sedan', 19940), ('Station Wagon/Sport Utility Vehicle', 16424), ('Pick-Up Truck', 995), ('Taxi', 812), ('Box Truck', 442), ('Bus', 268), ('Tractor Truck Diesel', 201), ('Bike', 197), ('Van', 187), ('Motorcycle', 169), ('Convertible', 119), ('E-Bike', 64), ('Flat Bed', 60), ('E-Scooter', 58), ('Dump', 56), ('Ambulance', 55), ('Moped', 50), ('Garbage Or Refuse', 47)]


In [16]:
#aca igual
codigo_tipo_de_vehiculo_3_counter = ctr(codigo_tipo_de_vehiculo_3)
codigo_tipo_de_vehiculo_3_most_common = codigo_tipo_de_vehiculo_3_counter.most_common(50)
print(codigo_tipo_de_vehiculo_3_most_common)

[('Nan', 460565), ('Sedan', 19915), ('Station Wagon/Sport Utility Vehicle', 16399), ('Pick-Up Truck', 975), ('Taxi', 789), ('Box Truck', 423), ('Bus', 248), ('Tractor Truck Diesel', 173), ('Bike', 172), ('Van', 171), ('Motorcycle', 144), ('Convertible', 102), ('Dump', 42), ('Pk', 42), ('Flat Bed', 40), ('Ambulance', 37), ('E-Bike', 37), ('Tow Truck / Wrecker', 36), ('E-Scooter', 34), ('Garbage Or Refuse', 28), ('4 Dr Sedan', 26), ('Moped', 26), ('Tractor Truck Gasoline', 19), ('Trailer', 15), ('Carry All', 15), ('Motorbike', 14), ('Motorscooter', 14), ('Limo', 12), ('Concrete Mixer', 10), ('Chassis Cab', 8), ('Tanker', 7), ('Refrigerated Van', 6), ('Truck', 6), ('School Bus', 6), ('Unk', 5), ('Lift Boom', 5), ('3-Door', 5), ('Unknown', 5), ('Multi-Wheeled Vehicle', 5), ('Bulk Agriculture', 4), ('Tow Truck', 4), ('Flat Rack', 4), ('Stake Or Rack', 4), ('E-Bik', 4), ('Ambul', 4), ('Trail', 4), ('Garbage Tr', 3), ('Fdny Fire', 3), ('Open Body', 3), ('Rv', 3)]


In [17]:
#una vez checkeado que todo esta bien reemplazo las columnas viejas por las limpias
choques_nyc['codigo_tipo_de_vehiculo_1'] = vehiculo_1
choques_nyc['codigo_tipo_de_vehiculo_2'] = vehiculo_2
choques_nyc['codigo_tipo_de_vehiculo_3'] = vehiculo_3
choques_nyc['codigo_tipo_de_vehiculo_4'] = vehiculo_4
choques_nyc['codigo_tipo_de_vehiculo_5'] = vehiculo_5

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\4201526722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['codigo_tipo_de_vehiculo_1'] = vehiculo_1
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\4201526722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['codigo_tipo_de_vehiculo_2'] = vehiculo_2
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\4201526722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [18]:
#replazando nan por Sin especificar en cada columna de factor contribuyente
choques_nyc.factor_contribuyente_vehiculo_1.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_2.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_3.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_4.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_5.fillna('Sin Especificar', inplace= True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2178874517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_1.fillna('Sin Especificar', inplace= True)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2178874517.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_2.fillna('Sin Especificar', inplace= True)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2178874517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [19]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_1.replace(['Following Too Closely',
'Unspecified',
'Turning Improperly',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Steering Failure',
'Traffic Control Disregarded',
'Drugs (illegal)',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Other Lighting Defects',
'Oversized Vehicle',
'Animals Action',
'Outside Car Distraction',
'Illnes',
'Driverless/Runaway Vehicle',
'Passenger Distraction',
'Tire Failure/Inadequate',
'Fatigued/Drowsy',
'Glare',
'Cell Phone (hand-Held)',
'Obstruction/Debris',
'Shoulders Defective/Improper',
'Lost Consciousness',
'Accelerator Defective',
'Failure to Keep Right',
'Traffic Control Device Improper/Non-Working',
'Eating or Drinking',
'Vehicle Vandalism',
'Physical Disability',
'Cell Phone (hands-free)',
'Lane Marking Improper/Inadequate',
'Other Electronic Device',
'Using On Board Navigation Device',
'Tow Hitch Defective',
'Texting',
'Headlights Defective',
'Tinted Windows',
'Prescription Medication',
'Listening/Using Headphones',
'Windshield Inadequate'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Giro Inapropiado',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Falla Al Doblar',
'Ignorar Control De Trafico',
'Drogas (ilegales)',
'Conduccion Agresiva',
'Quedarse Dormido',
'Peaton/Ciclista/Otro Error De Peaton/Confusion',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Otros Defectos De Iluminacion',
'Vehiculo Demasiado Grande',
'Accion Animal',
'Distraccion De Auto Ajeno',
'Enfermedad',
'Vehiculo Sin Conductor',
'Distraccion De Pasajero',
'Falla/Defecto de neumatico',
'Fatiga/Adormecimiento',
'Encandilamiento',
'Celular(En Mano)',
'Obstruccion/Escombros',
'Guardabarro Defectivo/Inapropiado',
'Perdida De Conciencia',
'Acelerador Defectuoso',
'Fallo De Mantener En Linear Recta',
'Dispositivo De Control De Trafico Roto/Inapropiado',
'Comida O Bebida',
'Vandalismo De Vehiculo',
'Discapacidad Fisica',
'Celular(Manos Libres)',
'Delimitacion de Carrilo Inadecuado',
'Otro Dispositivo Electronico',
'Usar Dispositivo De Navigacion De A Bordo',
'Enganche De Remolque Defectuoso',
'Mensajeria',
'Luces Defectivas',
'Vidrios Polarizados',
'Prescripcion Medica',
'Usar Auriculares',
'Parabrisas Inadecuados'], inplace= True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\1851416316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_1.replace(['Following Too Closely',


In [20]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_2.replace(['Following Too Closely',
'Unspecified',
'Turning Improperly',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Steering Failure',
'Traffic Control Disregarded',
'Drugs (illegal)',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Other Lighting Defects',
'Oversized Vehicle',
'Animals Action',
'Outside Car Distraction',
'Illnes',
'Driverless/Runaway Vehicle',
'Passenger Distraction',
'Tire Failure/Inadequate',
'Fatigued/Drowsy',
'Glare',
'Cell Phone (hand-Held)',
'Obstruction/Debris',
'Shoulders Defective/Improper',
'Lost Consciousness',
'Accelerator Defective',
'Failure to Keep Right',
'Traffic Control Device Improper/Non-Working',
'Eating or Drinking',
'Vehicle Vandalism',
'Physical Disability',
'Cell Phone (hands-free)',
'Lane Marking Improper/Inadequate',
'Other Electronic Device',
'Using On Board Navigation Device',
'Tow Hitch Defective',
'Texting',
'Headlights Defective',
'Tinted Windows',
'Prescription Medication',
'Listening/Using Headphones'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Giro Inapropiado',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Falla Al Doblar',
'Ignorar Control De Trafico',
'Drogas (ilegales)',
'Conduccion Agresiva',
'Quedarse Dormido',
'Peaton/Ciclista/Otro Error De Peaton/Confusion',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Otros Defectos De Iluminacion',
'Vehiculo Demasiado Grande',
'Accion Animal',
'Distraccion De Auto Ajeno',
'Enfermedad',
'Vehiculo Sin Conductor',
'Distraccion De Pasajero',
'Falla/Defecto de neumatico',
'Fatiga/Adormecimiento',
'Encandilamiento',
'Celular(En Mano)',
'Obstruccion/Escombros',
'Guardabarro Defectivo/Inapropiado',
'Perdida De Conciencia',
'Acelerador Defectuoso',
'Fallo De Mantener En Linea Recta',
'Dispositivo De Control De Trafico Roto/Inapropiado',
'Comida O Bebida',
'Vandalismo De Vehiculo',
'Discapacidad Fisica',
'Celular(Manos Libres)',
'Delimitacion de Carrilo Inadecuado',
'Otro Dispositivo Electronico',
'Usar Dispositivo De Navigacion De A Bordo',
'Enganche De Remolque Defectuoso',
'Mensajeria',
'Luces Defectivas',
'Vidrios Polarizados',
'Prescripcion Medica',
'Usar Auriculares'], inplace= True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\3914632134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_2.replace(['Following Too Closely',


In [21]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_3.replace(['Following Too Closely',
'Unspecified',
'Turning Improperly',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Steering Failure',
'Traffic Control Disregarded',
'Drugs (illegal)',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Oversized Vehicle',
'Outside Car Distraction',
'Illnes',
'Driverless/Runaway Vehicle',
'Passenger Distraction',
'Tire Failure/Inadequate',
'Glare',
'Obstruction/Debris',
'Lost Consciousness',
'Failure to Keep Right',
'Traffic Control Device Improper/Non-Working',
'Eating or Drinking',
'Cell Phone (hands-free)',
'Other Electronic Device'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Giro Inapropiado',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Falla Al Doblar',
'Ignorar Control De Trafico',
'Drogas (ilegales)',
'Conduccion Agresiva',
'Quedarse Dormido',
'Peaton/Ciclista/Otro Error De Peaton/Confusion',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Vehiculo Demasiado Grande',
'Distraccion De Auto Ajeno',
'Enfermedad',
'Vehiculo Sin Conductor',
'Distraccion De Pasajero',
'Falla/Defecto de neumatico',
'Encandilamiento',
'Obstruccion/Escombros',
'Perdida De Conciencia',
'Fallo De Mantener En Linear Recta',
'Dispositivo De Control De Trafico Roto/Inapropiado',
'Comida O Bebida',
'Celular(Manos Libres)',
'Otro Dispositivo Electronico'], inplace= True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\582211393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_3.replace(['Following Too Closely',


In [22]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_4.replace(['Following Too Closely',
'Unspecified',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Traffic Control Disregarded',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Outside Car Distraction',
'Obstruction/Debris'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Ignorar Control De Trafico',
'Conduccion Agresiva',
'Quedarse Dormido',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Distraccion De Auto Ajeno',
'Obstruccion/Escombros'], inplace= True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\3774588508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_4.replace(['Following Too Closely',


In [23]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_5.replace(['Following Too Closely',
'Unspecified',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Unsafe Speed',
'Reaction to Uninvolved Vehicle',
'Alcohol Involvement',
'Pavement Defective',
'Outside Car Distraction',
'Obstruction/Debris'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Velocidad Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Alcohol Involucrado',
'Pavimento defectuoso',
'Distraccion De Auto Ajeno',
'Obstruccion/Escombros'], inplace= True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2432672975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_5.replace(['Following Too Closely',


In [24]:
choques_nyc.factor_contribuyente_vehiculo_1.unique()

array(['Reversa Peligrosa', 'Velocidad Peligrosa', 'Seguir Muy De Cerca',
       'Sin Especificar', 'Pavimento Resbaladizo',
       'Obstruccion/Escombros', 'Pasar O Usar carril Inapropiadamente',
       'Conductor Inatento/Distraido', 'Otra Vehicular',
       'Pasar Muy De Cerca', 'Cambio de carril peligroso',
       'Inexperiencia Del Conductor', 'No Otorgar Derecho De Paso',
       'Frenos Defectuosos', 'Giro Inapropiado',
       'Reaccion A Vehiculo No Involucrado', 'Vision Obstruida/Limitada',
       'Falla Al Doblar', 'Ignorar Control De Trafico',
       'Drogas (ilegales)', 'Conduccion Agresiva', 'Quedarse Dormido',
       'Peaton/Ciclista/Otro Error De Peaton/Confusion',
       'Alcohol Involucrado', 'Pavimento defectuoso',
       'Otros Defectos De Iluminacion', 'Vehiculo Demasiado Grande',
       'Accion Animal', 'Distraccion De Auto Ajeno', 'Enfermedad',
       'Vehiculo Sin Conductor', 'Distraccion De Pasajero',
       'Falla/Defecto de neumatico', 'Fatiga/Adormecimiento',


In [25]:
choques_nyc.factor_contribuyente_vehiculo_2.unique()

array(['Sin Especificar', 'Pasar O Usar carril Inapropiadamente',
       'Otra Vehicular', 'Conductor Inatento/Distraido',
       'Velocidad Peligrosa', 'Pasar Muy De Cerca',
       'Inexperiencia Del Conductor', 'No Otorgar Derecho De Paso',
       'Seguir Muy De Cerca', 'Giro Inapropiado',
       'Vision Obstruida/Limitada', 'Vehiculo Demasiado Grande',
       'Cambio de carril peligroso', 'Ignorar Control De Trafico',
       'Pavimento Resbaladizo', 'Reaccion A Vehiculo No Involucrado',
       'Peaton/Ciclista/Otro Error De Peaton/Confusion',
       'Alcohol Involucrado', 'Quedarse Dormido', 'Conduccion Agresiva',
       'Dispositivo De Control De Trafico Roto/Inapropiado',
       'Encandilamiento', 'Reversa Peligrosa',
       'Distraccion De Auto Ajeno', 'Obstruccion/Escombros',
       'Fatiga/Adormecimiento', 'Vidrios Polarizados',
       'Vandalismo De Vehiculo', 'Frenos Defectuosos',
       'Distraccion De Pasajero', 'Falla Al Doblar',
       'Falla/Defecto de neumatico', 'Fallo

In [26]:
choques_nyc.factor_contribuyente_vehiculo_3.unique()

array(['Sin Especificar', 'Pasar O Usar carril Inapropiadamente',
       'Seguir Muy De Cerca', 'Pavimento Resbaladizo', 'Otra Vehicular',
       'Cambio de carril peligroso', 'Vision Obstruida/Limitada',
       'Distraccion De Auto Ajeno', 'No Otorgar Derecho De Paso',
       'Reaccion A Vehiculo No Involucrado',
       'Conductor Inatento/Distraido', 'Conduccion Agresiva',
       'Reversa Peligrosa', 'Ignorar Control De Trafico',
       'Vehiculo Demasiado Grande', 'Velocidad Peligrosa',
       'Inexperiencia Del Conductor', 'Obstruccion/Escombros',
       'Peaton/Ciclista/Otro Error De Peaton/Confusion',
       'Pasar Muy De Cerca', 'Alcohol Involucrado',
       'Dispositivo De Control De Trafico Roto/Inapropiado',
       'Pavimento defectuoso', 'Falla/Defecto de neumatico',
       'Quedarse Dormido', 'Giro Inapropiado',
       'Fallo De Mantener En Linear Recta', 'Celular(Manos Libres)',
       'Vehiculo Sin Conductor', 'Otro Dispositivo Electronico',
       'Distraccion De Pasajer

In [27]:
choques_nyc.factor_contribuyente_vehiculo_4.unique()

array(['Sin Especificar', 'Otra Vehicular',
       'Reaccion A Vehiculo No Involucrado', 'Seguir Muy De Cerca',
       'Conductor Inatento/Distraido', 'Alcohol Involucrado',
       'Inexperiencia Del Conductor',
       'Pasar O Usar carril Inapropiadamente',
       'Distraccion De Auto Ajeno', 'Frenos Defectuosos',
       'Pavimento Resbaladizo', 'Obstruccion/Escombros',
       'Pasar Muy De Cerca', 'Ignorar Control De Trafico',
       'Vision Obstruida/Limitada', 'Quedarse Dormido',
       'No Otorgar Derecho De Paso', 'Velocidad Peligrosa',
       'Reversa Peligrosa', 'Conduccion Agresiva',
       'Cambio de carril peligroso', 'Pavimento defectuoso'], dtype=object)

In [28]:
choques_nyc.factor_contribuyente_vehiculo_5.unique()

array(['Sin Especificar', 'Alcohol Involucrado',
       'Reaccion A Vehiculo No Involucrado', 'Seguir Muy De Cerca',
       'Conductor Inatento/Distraido', 'Otra Vehicular',
       'Pasar Muy De Cerca', 'Distraccion De Auto Ajeno',
       'Inexperiencia Del Conductor', 'Pavimento Resbaladizo',
       'Obstruccion/Escombros', 'Velocidad Peligrosa',
       'No Otorgar Derecho De Paso', 'Pavimento defectuoso'], dtype=object)

In [29]:
choques_nyc.factor_contribuyente_vehiculo_5.unique()

array(['Sin Especificar', 'Alcohol Involucrado',
       'Reaccion A Vehiculo No Involucrado', 'Seguir Muy De Cerca',
       'Conductor Inatento/Distraido', 'Otra Vehicular',
       'Pasar Muy De Cerca', 'Distraccion De Auto Ajeno',
       'Inexperiencia Del Conductor', 'Pavimento Resbaladizo',
       'Obstruccion/Escombros', 'Velocidad Peligrosa',
       'No Otorgar Derecho De Paso', 'Pavimento defectuoso'], dtype=object)

In [30]:
choques_nyc.codigo_tipo_de_vehiculo_1.unique()

array(['Sedan', 'Station Wagon/Sport Utility Vehicle', 'Bus', 'Bike',
       'Taxi', 'Motorcycle', 'Dump', 'Pick-Up Truck', 'Box Truck',
       'Unknown', 'Ambulance', 'E-Scooter', 'E-Bike', 'Van', 'Flat Bed',
       'Moped', 'Tractor Truck Diesel', 'Garbage Or Refuse',
       'Convertible'], dtype=object)

In [31]:
choques_nyc.codigo_tipo_de_vehiculo_1.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\206431653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_1.replace(


In [32]:
choques_nyc.codigo_tipo_de_vehiculo_2.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\51666981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_2.replace(


In [33]:
choques_nyc.codigo_tipo_de_vehiculo_3.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\830710450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_3.replace(


In [34]:
choques_nyc.codigo_tipo_de_vehiculo_4.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\439636689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_4.replace(


In [35]:
choques_nyc.codigo_tipo_de_vehiculo_5.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\1373640272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_5.replace(


In [36]:
choques_nyc.codigo_tipo_de_vehiculo_1.unique()

array(['Sedan', 'Camioneta/Vehiculo Utilitario Deportivo', 'Autobus',
       'Motocicleta', 'Taxi', 'Camion De Basura', 'Camioneta',
       'Camion De Caja', 'Desconocido', 'Ambulancia', 'E-Scooter',
       'E-Bike', 'Furgoneta', 'Camion', 'Tractor', 'Convertible'],
      dtype=object)

In [37]:
choques_nyc.columns

Index(['colision_fecha', 'colision_hora', 'latitud', 'longitud', 'ubicacion',
       'nombre_de_calle', 'numero_de_personas_heridas',
       'numero_de_personas_fallecidas', 'numero_de_peatones_heridos',
       'numero_de_peatones_fallecidos', 'numero_de_ciclistas_heridos',
       'numero_de_ciclistas_fallecidos', 'numero_de_conductores_heridos',
       'numero_de_conductores_fallecidos', 'factor_contribuyente_vehiculo_1',
       'factor_contribuyente_vehiculo_2', 'colision_ID',
       'codigo_tipo_de_vehiculo_1', 'codigo_tipo_de_vehiculo_2', 'localidad',
       'codigo_postal', 'calle_interseccion', 'nombre_fuera_de_calle',
       'factor_contribuyente_vehiculo_3', 'factor_contribuyente_vehiculo_4',
       'codigo_tipo_de_vehiculo_3', 'codigo_tipo_de_vehiculo_4',
       'factor_contribuyente_vehiculo_5', 'codigo_tipo_de_vehiculo_5',
       'colision_fecha_hora', 'anio', 'mes', 'dia', 'hora'],
      dtype='object')

In [38]:
choques_nyc['tipo_causante']= choques_nyc['codigo_tipo_de_vehiculo_1']

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2697252169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['tipo_causante']= choques_nyc['codigo_tipo_de_vehiculo_1']


In [39]:
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_1 != 'Desconocido'),'codigo_tipo_de_vehiculo_1']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_2 != 'Desconocido'),'codigo_tipo_de_vehiculo_2']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_3 != 'Desconocido'),'codigo_tipo_de_vehiculo_3']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_4 != 'Desconocido'),'codigo_tipo_de_vehiculo_4']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_5 != 'Desconocido'),'codigo_tipo_de_vehiculo_5']=1

In [40]:
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_1 == 'Desconocido'),'codigo_tipo_de_vehiculo_1']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_2 == 'Desconocido'),'codigo_tipo_de_vehiculo_2']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_3 == 'Desconocido'),'codigo_tipo_de_vehiculo_3']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_4 == 'Desconocido'),'codigo_tipo_de_vehiculo_4']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_5 == 'Desconocido'),'codigo_tipo_de_vehiculo_5']=0

In [41]:
choques_nyc['cantidad_autos_choque']= choques_nyc['codigo_tipo_de_vehiculo_1']+choques_nyc['codigo_tipo_de_vehiculo_2'] + choques_nyc['codigo_tipo_de_vehiculo_3'] + choques_nyc['codigo_tipo_de_vehiculo_4'] + choques_nyc['codigo_tipo_de_vehiculo_5']

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\2346095221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['cantidad_autos_choque']= choques_nyc['codigo_tipo_de_vehiculo_1']+choques_nyc['codigo_tipo_de_vehiculo_2'] + choques_nyc['codigo_tipo_de_vehiculo_3'] + choques_nyc['codigo_tipo_de_vehiculo_4'] + choques_nyc['codigo_tipo_de_vehiculo_5']


In [42]:
choques_nyc['numero_de_personas_heridas'].fillna(0, inplace=True)
choques_nyc['numero_de_personas_fallecidas'].fillna(0, inplace=True)
choques_nyc['numero_de_peatones_heridos'].fillna(0, inplace=True)
choques_nyc['numero_de_peatones_fallecidos'].fillna(0, inplace=True)
choques_nyc['numero_de_ciclistas_heridos'].fillna(0, inplace=True)
choques_nyc['numero_de_ciclistas_fallecidos'].fillna(0, inplace=True)
choques_nyc['numero_de_conductores_heridos'].fillna(0, inplace=True)
choques_nyc['numero_de_conductores_fallecidos'].fillna(0, inplace=True)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\1774733682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_heridas'].fillna(0, inplace=True)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\1774733682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_fallecidas'].fillna(0, inplace=True)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\1774733682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [43]:
choques_nyc['numero_de_personas_heridas']=choques_nyc['numero_de_personas_heridas'].astype(int)
choques_nyc['numero_de_personas_fallecidas']= choques_nyc['numero_de_personas_fallecidas'].astype(int)
choques_nyc['numero_de_peatones_heridos']= choques_nyc['numero_de_peatones_heridos'].astype(int)
choques_nyc['numero_de_peatones_fallecidos']= choques_nyc['numero_de_peatones_fallecidos'].astype(int)
choques_nyc['numero_de_ciclistas_heridos']= choques_nyc['numero_de_ciclistas_heridos'].astype(int)
choques_nyc['numero_de_ciclistas_fallecidos']= choques_nyc['numero_de_ciclistas_fallecidos'].astype(int)
choques_nyc['numero_de_conductores_heridos']= choques_nyc['numero_de_conductores_heridos'].astype(int)
choques_nyc['numero_de_conductores_fallecidos']= choques_nyc['numero_de_conductores_fallecidos'].astype(int)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\3398349128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_heridas']=choques_nyc['numero_de_personas_heridas'].astype(int)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\3398349128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_fallecidas']= choques_nyc['numero_de_personas_fallecidas'].astype(int)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\3398349128.py:3: Se

In [44]:
choques_nyc['numero_de_personas_fallecidas'] = choques_nyc['numero_de_personas_fallecidas'].apply(lambda x: x*2)
choques_nyc['numero_de_peatones_fallecidos'] = choques_nyc['numero_de_peatones_fallecidos'].apply(lambda x: x*2)
choques_nyc['numero_de_ciclistas_fallecidos'] = choques_nyc['numero_de_ciclistas_fallecidos'].apply(lambda x: x*2)
choques_nyc['numero_de_conductores_fallecidos'] = choques_nyc['numero_de_conductores_fallecidos'].apply(lambda x: x*2)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\146870644.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_fallecidas'] = choques_nyc['numero_de_personas_fallecidas'].apply(lambda x: x*2)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\146870644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_peatones_fallecidos'] = choques_nyc['numero_de_peatones_fallecidos'].apply(lambda x: x*2)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_

In [45]:
columnas=['numero_de_personas_fallecidas', 'numero_de_peatones_fallecidos', 'numero_de_ciclistas_fallecidos', 'numero_de_conductores_fallecidos', 'numero_de_personas_heridas', 'numero_de_peatones_heridos', 'numero_de_ciclistas_heridos', 'numero_de_conductores_heridos']

In [46]:
choques_nyc['letalidad']= choques_nyc[columnas].sum(axis=1)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\467996619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['letalidad']= choques_nyc[columnas].sum(axis=1)


In [47]:
choques_nyc.numero_de_conductores_fallecidos.unique()

array([0, 2, 4, 6, 8], dtype=int64)

In [48]:
choques_nyc['latitud']= choques_nyc['latitud'].astype(float)
choques_nyc['longitud']= choques_nyc['longitud'].astype(float)

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\244482025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['latitud']= choques_nyc['latitud'].astype(float)
C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\244482025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['longitud']= choques_nyc['longitud'].astype(float)


In [49]:
choques_nyc=choques_nyc.dropna(subset=['latitud', 'longitud']).reset_index(drop=True)

In [50]:
prototipo= pd.DataFrame().assign(id_choque=choques_nyc['colision_ID'],fecha_y_hora= choques_nyc['colision_fecha_hora'],anio= choques_nyc['anio'],mes=choques_nyc['mes'],dia=choques_nyc['dia'],latitud=choques_nyc['latitud'], longitud=choques_nyc['longitud'], auto_causante=choques_nyc['tipo_causante'],localidad=choques_nyc['localidad'],codigo_potal=choques_nyc['codigo_postal'], cantidad_autos_choque=choques_nyc['cantidad_autos_choque'],letalidad=choques_nyc['letalidad'] )

In [51]:
prototipo = prototipo.set_index('id_choque')

In [52]:
principal_sql=prototipo 
principal_sql.drop('fecha_y_hora', axis=1, inplace=True)

In [53]:
prototipo.columns

Index(['anio', 'mes', 'dia', 'latitud', 'longitud', 'auto_causante',
       'localidad', 'codigo_potal', 'cantidad_autos_choque', 'letalidad'],
      dtype='object')

In [54]:
principal_sql.columns

Index(['anio', 'mes', 'dia', 'latitud', 'longitud', 'auto_causante',
       'localidad', 'codigo_potal', 'cantidad_autos_choque', 'letalidad'],
      dtype='object')

In [55]:
prototipo.auto_causante.unique()

array(['Sedan', 'Camioneta/Vehiculo Utilitario Deportivo',
       'Camion De Basura', 'Camioneta', 'Camion De Caja', 'Desconocido',
       'Ambulancia', 'E-Scooter', 'Taxi', 'E-Bike', 'Furgoneta', 'Camion',
       'Motocicleta', 'Tractor', 'Autobus', 'Convertible'], dtype=object)

In [56]:
principal_sql.auto_causante.unique()

array(['Sedan', 'Camioneta/Vehiculo Utilitario Deportivo',
       'Camion De Basura', 'Camioneta', 'Camion De Caja', 'Desconocido',
       'Ambulancia', 'E-Scooter', 'Taxi', 'E-Bike', 'Furgoneta', 'Camion',
       'Motocicleta', 'Tractor', 'Autobus', 'Convertible'], dtype=object)

In [57]:
prototipo['auto_causante']=prototipo['auto_causante'].str.replace('[^\w\s]', '')

C:\Users\Alfredo BTP\AppData\Local\Temp\ipykernel_21436\1350600759.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  prototipo['auto_causante']=prototipo['auto_causante'].str.replace('[^\w\s]', '')


In [58]:
prototipo.head(10)

,anio,mes,dia,latitud,longitud,auto_causante,localidad,codigo_potal,cantidad_autos_choque,letalidad
id_choque,,,,,,,,,,
4547589,2022,7,20,40.835808,-73.890830,Sedan,NaN,NaN,2,0
4407147,2021,4,13,40.683580,-73.976170,Sedan,BROOKLYN,11217,1,2
4406885,2021,4,12,0.000000,0.000000,CamionetaVehiculo Utilitario Deportivo,NaN,NaN,2,0
4548548,2022,7,21,40.715070,-73.913270,Sedan,NaN,NaN,2,2
4136992,2019,5,21,40.697540,-73.983120,Camion De Basura,BROOKLYN,11201,2,0
4395664,2021,2,26,40.843464,-73.836000,CamionetaVehiculo Utilitario Deportivo,BRONX,10461,1,0
4397513,2021,3,9,40.692547,-73.990974,Camioneta,NaN,NaN,2,2
4403773,2021,3,31,40.626457,-73.918000,Sedan,BROOKLYN,11234,2,2
4405244,2021,4,6,40.526894,-74.167280,CamionetaVehiculo Utilitario Deportivo,STATEN ISLAND,10312,2,14


In [59]:
results_df_robo['cmplnt_to_dt'].replace({'T00:00:00.000':''}, regex=True, inplace=True)
results_df_robo['fecha_y_hora']=pd.to_datetime(results_df_robo['cmplnt_to_dt'] + results_df_robo['cmplnt_fr_tm'], format='%Y-%m-%d%H:%M:%S')

In [60]:
crimen=results_df_robo[results_df_robo['fecha_y_hora'].dt.year.isin([2019,2020,2021,2022])]

In [61]:
robo= pd.DataFrame().assign(fecha_y_hora= crimen['fecha_y_hora'], delito= crimen['pd_desc'], localidad= crimen['boro_nm'],latitud= crimen['latitude'],longitud= crimen['longitude'],)

In [62]:
robo['anio'] = robo['fecha_y_hora'].dt.year
robo['mes'] = robo['fecha_y_hora'].dt.month
robo['dia'] = robo['fecha_y_hora'].dt.day
robo['hora'] = robo['fecha_y_hora'].dt.time

In [63]:
robo= robo.drop(robo[robo.delito != 'ROBBERY,CAR JACKING'].index).reset_index(drop=True)

In [64]:
robo_sql=robo
robo_sql.drop('fecha_y_hora', axis=1, inplace=True)
robo_sql.drop('hora', axis=1, inplace=True)

In [65]:
results_df_edad['crash_date'].replace({'T00:00:00.000':''}, regex=True, inplace=True)

In [66]:
results_df_edad['crash_date_time']=pd.to_datetime(results_df_edad['crash_date'] + results_df_edad['crash_time'], format='%Y-%m-%d%H:%M')

In [67]:
choques_df=results_df_edad[results_df_edad['crash_date_time'].dt.year.isin([2019,2020,2021,2022])].reset_index(drop=True)

In [68]:
causa_df = pd.DataFrame().assign(id_choque=choques_df['collision_id'], fecha_hora=choques_df['crash_date_time'], sexo=choques_df['driver_sex'],anio_del_vehiculo=choques_df['vehicle_year'])

In [69]:
causa_df['anio'] = causa_df['fecha_hora'].dt.year
causa_df['mes'] = causa_df['fecha_hora'].dt.month
causa_df['dia'] = causa_df['fecha_hora'].dt.day
causa_df['hora'] = causa_df['fecha_hora'].dt.time

In [70]:
causa_df = causa_df.set_index('id_choque')

In [72]:
results_edad_2 = client_edad.get("f55k-p6yu", limit=1000000)
results_df_edad_2 = pd.DataFrame.from_records(results_edad_2)

In [73]:
results_df_edad_2['crash_date'].replace({'T00:00:00.000':''}, regex=True, inplace=True)

In [74]:
results_df_edad_2['crash_date_time']=pd.to_datetime(results_df_edad_2['crash_date'] + results_df_edad_2['crash_time'], format='%Y-%m-%d%H:%M')

In [75]:
personas_df=results_df_edad_2[results_df_edad_2['crash_date_time'].dt.year.isin([2019,2020,2021,2022])].reset_index(drop=True)

In [76]:
edad = pd.DataFrame().assign(id_choque=personas_df['collision_id'], edad=personas_df['person_age'])

In [77]:
edad=edad.dropna(subset=['edad']).reset_index(drop=True)

In [78]:
edad['edad'] = edad['edad'].astype(int)

In [79]:
edad = edad.drop(edad[edad.edad > 80].index).reset_index(drop=True)
edad = edad.drop(edad[edad.edad < 16].index).reset_index(drop=True)

In [80]:
bins= [0,16,22,36,51,102]
labels = ['menor','adolescente','adulto_joven','adulto','adulto_mayor']
edad['rango_etario'] = pd.cut(edad['edad'], bins=bins, labels=labels, right=False)

In [81]:
causa2=causa_df

In [82]:
causa2.drop('fecha_hora', axis=1, inplace=True)
causa2.drop('hora', axis=1, inplace=True)

In [83]:
prototipo.to_csv('principal_sql.csv')
robo_sql.to_csv('robo_sql.csv')
edad.to_csv('edad_sql.csv')
causa2.to_csv('causa_sql.csv')